### BIBLIOTECAS

In [9]:
import os
import pandas as pd
import numpy as np
import glob

### LEITURA DOS ARQUIVOS

In [10]:
def ler_dados_pickle(caminho_arquivo_pkl):
    """Carrega os dados de um arquivo .pkl em um DataFrame."""
    return pd.read_pickle(caminho_arquivo_pkl)

### Nova Chamada 1: Carregar dados de todos os arquivos .pkl no diretório ###
caminho_arquivos_pkl = glob.glob(r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Entrada\*.pkl')

# Definir a pasta de saída desejada
pasta_saida_desejada = r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_com_NaNs_preenchidos'

### FUNÇÕES

In [11]:
### FUNÇÕES

### FUNÇÃO 2: Deletar colunas específicas do DataFrame ###
def deletar_colunas(df):
    """Deleta colunas específicas do DataFrame."""
    colunas = [x for x in range(df.shape[1])]  # list of columns' integer indices
    colunas_para_deletar = [2, 3, 24, 26, 27, 28, 42, 43] # colunas a serem removidas
    for coluna_para_deletar in colunas_para_deletar:
        colunas.remove(coluna_para_deletar)
    return df.iloc[:, colunas]

### FUNÇÃO 3: Substituir valores comentados por NaN ###
def substituir_valores_comentados_por_nan(df):
    """Substitui os valores que começam com '#' por NaN em todo o DataFrame."""
    return df.replace(to_replace=r'^#.*', value=np.nan, regex=True)

### FUNÇÃO 4: Substituir virgulas por pontos
def substituir_virgulas_por_pontos(df):
    df = df.applymap(lambda x: str(x).replace(',', '.'))
    return df

### FUNÇÃO 5: Converter strings que representam NaN para NaN numérico ###
def converter_str_nan_para_numerico(df):
    """Converte strings que representam NaN para NaN numérico."""
    return df.replace({'nan': np.nan, 'NaN': np.nan, 'NAN': np.nan})

### FUNÇÃO 6: Preencher os NaN em cada coluna com a média dos valores anteriores e posteriores ###
def preencher_nans_com_media(df, colunas_para_preencher):
    """Preenche os NaN em cada coluna com a média dos valores anteriores e posteriores."""
    for coluna in colunas_para_preencher:
        df[coluna] = pd.to_numeric(df[coluna], errors='coerce')
        df[coluna] = df[coluna].fillna((df[coluna].shift() + df[coluna].shift(-1)) / 2)
    return df

### FUNÇÃO 7: Substituir NaNs por zero se o valor acima ou abaixo do NaN for zero ###
def substituir_nans_por_zero(df, colunas_para_preencher):
    """Substitui NaNs por 0 se o valor acima ou abaixo do NaN for 0."""
    for coluna in colunas_para_preencher:
        df[coluna] = pd.to_numeric(df[coluna], errors='coerce')
        indices_nan = df[coluna][df[coluna].isna()].index
        for indice_nan in indices_nan:
            if indice_nan > 0:
                ultimo_valor_anterior = df[coluna][indice_nan - 1]
                if pd.notna(ultimo_valor_anterior) and ultimo_valor_anterior == 0:
                    df.at[indice_nan, coluna] = 0
                    continue  # Passar para o próximo NaN se o valor anterior for zero
            if indice_nan < len(df[coluna]) - 1:
                proximo_valor_posterior = df[coluna][indice_nan + 1]
                if pd.notna(proximo_valor_posterior) and proximo_valor_posterior == 0:
                    df.at[indice_nan, coluna] = 0
    return df

### FUNÇÃO 8: Substituir NaNs pelo valor anterior se o valor acima ou abaixo do NaN for igual ###
def substituir_nans_com_valor_anterior_igual(df, colunas_para_preencher):
    """Substitui NaNs pelo valor anterior se o valor acima ou abaixo do NaN for igual."""
    for coluna in colunas_para_preencher:
        indices_nan = df[coluna][df[coluna].isna()].index
        for indice_nan in indices_nan:
            if indice_nan > 0:
                primeiro_valor_anterior = df[coluna][indice_nan - 1]
                if indice_nan < len(df[coluna]) - 1:
                    proximo_valor_posterior = df[coluna][indice_nan + 1]
                    if primeiro_valor_anterior == proximo_valor_posterior:
                        df.at[indice_nan, coluna] = primeiro_valor_anterior
    return df

### FUNÇÃO 9: Interpolar os NaNs em cada coluna com base nos valores imediatamente acima e abaixo ###
def interpolar_nans(df, colunas_para_preencher):
    """Interpola os NaNs em cada coluna com base nos valores imediatamente acima e abaixo."""
    for coluna in colunas_para_preencher:
        df[coluna] = df[coluna].interpolate(method='linear')
    return df

### FUNÇÃO 10: Remover duplicados
def remover_duplicatas(df, colunas_chave):
    """Remove linhas duplicadas com base em colunas específicas."""
    df.drop_duplicates(subset=colunas_chave, inplace=True)
    return df

### FUNÇÃO 11: Salvar o DataFrame como um arquivo pickle e, opcionalmente, um arquivo CSV ###
def salvar_arquivos(df, caminho_arquivo_pkl, pasta_saida, save_csv=False):
    """Salva o DataFrame como um arquivo pickle e, opcionalmente, um arquivo CSV."""
    caminho_arquivo_pkl_atualizado = os.path.join(pasta_saida, os.path.basename(caminho_arquivo_pkl).replace('.pkl', '_sem_missing_values.pkl'))

    # Salvar o arquivo pickle
    df.to_pickle(caminho_arquivo_pkl_atualizado)

    # Salvar o arquivo CSV, se a opção save_csv for True
    if save_csv:
        caminho_arquivo_csv = os.path.join(pasta_saida, os.path.basename(caminho_arquivo_pkl).replace('.pkl', '_sem_missing_values.csv'))
        df.to_csv(caminho_arquivo_csv, index=False)

### LEITURA DAS FUNÇÕES

In [12]:
# Iterar sobre cada caminho de arquivo
for caminho_arquivo_pkl in caminho_arquivos_pkl:
    # Ler os dados do arquivo .pkl
    df = ler_dados_pickle(caminho_arquivo_pkl)
    
    ### CHAMADA 2: Deletar colunas específicas do DataFrame ###
    df = deletar_colunas(df)

    ### CHAMADA 3: Substituir valores comentados por NaN ###
    df = substituir_valores_comentados_por_nan(df)

    ### CHAMADA 4: Substituir virgulas por pontos
    df = substituir_virgulas_por_pontos(df)

    ### CHAMADA 5: Converter strings NaN para NaN numérico ###
    df = converter_str_nan_para_numerico(df)

    ### CHAMADA 6: Preencher os NaN em todas as colunas do DataFrame, exceto as especificadas ###
    colunas_para_preencher = [coluna for coluna in df.columns if coluna not in ['index', 'Data', 'Hora']]
    df = preencher_nans_com_media(df, colunas_para_preencher)

    ### CHAMADA 7: Substituir NaNs por zero se o valor acima ou abaixo do NaN for zero ###
    df = substituir_nans_por_zero(df, colunas_para_preencher)

    ### CHAMADA 8: Substituir NaNs pelo valor anterior se o valor acima ou abaixo do NaN for igual ###
    df = substituir_nans_com_valor_anterior_igual(df, colunas_para_preencher)

    ### CHAMADA 9: Interpolar os NaNs com base nos valores imediatamente acima e abaixo ###
    df = interpolar_nans(df, colunas_para_preencher)

    ### CHAMADA 10: remover linhas duplicadas
    colunas_chave = ['Data', 'Hora']  # Colunas chave para identificar duplicatas
    df = remover_duplicatas(df, colunas_chave)

    # CHAMADA 11: Chamar a função para salvar os arquivos na pasta de saída desejada
    salvar_arquivos(df, caminho_arquivo_pkl, pasta_saida_desejada, save_csv=True)